In [1]:
import os 
from dotenv import load_dotenv, find_dotenv
from langchain.chains import ConversationChain
from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory

load_dotenv(find_dotenv())

True

In [2]:
# Inicialización del modelo de lenguaje y del buffer de conversación
llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"))
conversation_buffer = ConversationBufferMemory()

# Creación de la ConversationChain con el buffer de memoria

In [3]:
conversation_chain = ConversationChain(
    llm=llm, 
    memory=conversation_buffer
)

In [6]:
conversation_chain.memory.buffer

'Human: hello there\nAI: Hello! How can I assist you today?'

In [9]:
conversation_chain.memory.clear

<bound method BaseChatMemory.clear of ConversationBufferMemory()>

In [4]:
conversation_chain("hello there")

c:\Users\ignac\Documents\chat-with-rag-using-langchain-nextjs-and-fastapi\backend\venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'input': 'hello there',
 'history': '',
 'response': 'Hello! How can I assist you today?'}

In [16]:
conversation_chain("Im fine, thank yo")

{'input': 'Im fine, thank yo',
 'history': "Human: hello there\nAI: Hello! How can I assist you today?\nHuman: Im fine, thank yo\nAI: You're welcome! Is there anything specific you would like to chat about or any questions I can help answer?\nHuman: hello there\nAI: Hello! How can I assist you today? Is there anything specific you would like to chat about or any questions I can help answer?",
 'response': "You're welcome! I'm glad to hear that you're doing well. If you have any questions or if there's anything specific you would like to chat about, feel free to let me know."}

In [24]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")

In [25]:
len(enc.encode(conversation_buffer.buffer))

132

In [21]:
from langchain.memory import ChatMessageHistory
history = ChatMessageHistory()

In [22]:
history.add_messages(conversation_buffer)

In [23]:
history

ChatMessageHistory(messages=[('chat_memory', ChatMessageHistory(messages=[HumanMessage(content='hello there'), AIMessage(content='Hello! How can I assist you today?'), HumanMessage(content='Im fine, thank yo'), AIMessage(content="You're welcome! Is there anything specific you would like to chat about or any questions I can help answer?"), HumanMessage(content='hello there'), AIMessage(content='Hello! How can I assist you today? Is there anything specific you would like to chat about or any questions I can help answer?'), HumanMessage(content='Im fine, thank yo'), AIMessage(content="You're welcome! I'm glad to hear that you're doing well. If you have any questions or if there's anything specific you would like to chat about, feel free to let me know.")])), ('output_key', None), ('input_key', None), ('return_messages', False), ('human_prefix', 'Human'), ('ai_prefix', 'AI'), ('memory_key', 'history')])

In [ ]:
# Función para añadir un nuevo input y generar una respuesta
def ask_question(input_text):
    # Añadir el input al historial
    conversation_chain.conversation_buffer.add_user_input(input_text)
    
    # Generar la respuesta utilizando el historial actual
    response = conversation_chain.generate_response(input_text)
    
    # Añadir la respuesta al historial
    conversation_chain.conversation_buffer.add_model_response(response)
    
    return response

# Ejemplo de interacción
history = [
    {"input": "¿Cuál es la capital de Francia?", "docs": None},
    {"input": "¿Y qué famoso monumento se encuentra allí?", "docs": None}
]

# Pasar el historial al buffer de conversación
for item in history:
    conversation_chain.conversation_buffer.add_user_input(item["input"])
    # Aquí se omiten las respuestas del modelo para simplificar, pero normalmente se añadirían también.

# Hacer una nueva pregunta que dependa del contexto anterior
new_question = "¿Cuánto mide ese monumento?"
response = ask_question(new_question)

print(response)
